In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/understanding-unets


In [2]:
# # this just to make sure we are using only on CPU
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import time
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

from data import im_generators, ConcatenateGenerators
from evaluate import Metrics
from learned_wavelet import learned_wavelet
from unet import unet

Using TensorFlow backend.


In [4]:
np.random.seed(0)

In [5]:
plt.rcParams['figure.figsize'] = (9, 5)
plt.rcParams['image.cmap'] = 'gray'

In [6]:
im_gen_train, im_gen_val, im_gen_test, size, n_samples_train = im_generators(
    'bsd68', 
    batch_size=1, 
    validation_split=0.1, 
    no_augment=True, 
    noise_std=30,
)    
im_bsd68 = ConcatenateGenerators(im_gen_train, im_gen_val, im_gen_test)

In [7]:
all_net_params = [
    {
        'name': 'unet',
        'init_function': unet,
        'run_params': {
            'n_layers': 5, 
            'pool': 'max', 
            "layers_n_channels": [64, 128, 256, 512, 1024], 
            'layers_n_non_lins': 2,
            'non_relu_contract': False,
            'input_size': (size, size, 3),
        },
        'run_id': 'unet_div2k_30_1570541274',
    },
    {
        'name': 'learned wavelet',
        'init_function': learned_wavelet,
        'run_params': {
            'n_scales': 5, 
            'n_details': 64, 
            'n_coarse': 64, 
            'n_groupping': 64,
            'input_size': (size, size, 3),
        },
        'run_id': 'learned_wavelet_div2k_30_1570541994',
    },
]

In [8]:
def unpack_model(init_function=None, run_params=None, run_id=None, epoch=500, **dummy_kwargs):
    model = init_function(**run_params)
    chkpt_path = f'checkpoints/{run_id}-{epoch}.hdf5'
    model.load_weights(chkpt_path)
    return model

def enumerate_seq(seq, name):
    return (seq[i] for i in tqdm_notebook(range(len(seq)), desc=f'Val files for {name}'))

def metrics_for_params(reco_function=None, name=None, **net_params):
    model = unpack_model(**net_params)
    metrics = Metrics()
    pred_and_gt = [
        (model.predict_on_batch(images_noisy), images_gt)
        for images_noisy, images_gt in enumerate_seq(im_bsd68, name)
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc=f'Stats for {name}'):
        metrics.push(images, im_recos)
    return metrics

def metrics_original():
    metrics = Metrics()
    pred_and_gt = [
        (images_noisy, images_gt)
        for images_noisy, images_gt in enumerate_seq(im_bsd68, 'Original noisy image')
    ]    
    for im_recos, images in tqdm_notebook(pred_and_gt, desc='Original noisy image'):
        metrics.push(images, im_recos)
    return metrics

In [9]:
%%time
metrics = []
for net_params in all_net_params:
    metrics.append((net_params['name'], metrics_for_params(**net_params)))
    
metrics.append(('original', metrics_original()))

Instructions for updating:
Colocations handled automatically by placer.



CPU times: user 2min 5s, sys: 19.7 s, total: 2min 24s
Wall time: 2min 16s


In [10]:
metrics

[('unet', PSNR = 27.02 +/- 3.076 SSIM = 0.7347 +/- 0.09446),
 ('learned wavelet', PSNR = 26.98 +/- 3.339 SSIM = 0.7181 +/- 0.08254),
 ('original', PSNR = 19.37 +/- 0.8237 SSIM = 0.3535 +/- 0.2044)]

In [11]:
metrics.sort(key=lambda x: x[1].metrics['PSNR'].mean())

In [12]:
# import pickle
# with open('metrics_net_rec_fastmri', 'wb') as f:
#     pickle.dump(metrics, f)

In [13]:
def n_model_params_for_params(reco_function=None, val_gen=None, name=None, **net_params):
    model = unpack_model(**net_params)
    n_params = model.count_params()
    return n_params

In [14]:
%%time
n_params = {}
for net_params in all_net_params:
    n_params[net_params['name']] =  n_model_params_for_params(**net_params)
    
n_params['original'] =  0

CPU times: user 2 s, sys: 289 ms, total: 2.29 s
Wall time: 2.21 s


In [15]:
def runtime_for_params(reco_function=None, name=None, **net_params):
    model = unpack_model(**net_params)
    data = im_bsd68[0]
    start = time.time()
    model.predict_on_batch(data[0])
    end = time.time()
    return end - start

In [16]:
%%time
runtimes = {}
for net_params in tqdm_notebook(all_net_params):
    runtimes[net_params['name']] =  runtime_for_params(**net_params)
    
runtimes['original'] = 'NA'


CPU times: user 3 s, sys: 329 ms, total: 3.33 s
Wall time: 3.22 s


In [17]:
metrics_table = pd.DataFrame(
    index=[name for name, _ in metrics], 
    columns=['PSNR-mean (std) (dB)', 'SSIM-mean (std)', '# params', 'Runtime (s)'],
)
for name, m in metrics:
    metrics_table.loc[name, 'PSNR-mean (std) (dB)'] = "{mean:.4} ({std:.4})".format(
        mean=m.metrics['PSNR'].mean(), 
        std=m.metrics['PSNR'].stddev(),
    )
    metrics_table.loc[name, 'SSIM-mean (std)'] = "{mean:.4} ({std:.4})".format(
        mean=m.metrics['SSIM'].mean(), 
        std=m.metrics['SSIM'].stddev(),
    )
    metrics_table.loc[name, '# params'] = "{}".format(
        n_params[name], 
     )
    metrics_table.loc[name, 'Runtime (s)'] = "{runtime:.4}".format(
        runtime=runtimes[name], 
    )

In [18]:
metrics_table

,PSNR-mean (std) (dB),SSIM-mean (std),# params,Runtime (s)
original,19.37 (0.4118),0.3535 (0.1022),0,NA
learned wavelet,26.98 (1.669),0.7181 (0.04127),20460547,0.3183
unet,27.02 (1.538),0.7347 (0.04723),31032499,0.2544
